In [6]:
# Inputs:
dataset_file = 'dataset.csv'
local_kings_file = 'local_kings.csv'

# Outputs:
paired_dataset_file = 'paired_dataset.csv'

# Params:
min_delta_c = 20

In [7]:
import pandas as pd
import pickle
from multiprocessing import Pool

In [15]:
def assemble_pairs(k):

    # Read the pickle file for a given local king, here under the k variable
    filename = 'neighborhood_pairs/single_mutants_' + str(k) + '.pkl'
    with open(filename, 'rb') as f:
        nbh_pairs = pickle.load(f)

    # Assemble the results into a list
    part_res = []
    for idxs in nbh_pairs:
        
        i1, i2 = idxs
        r1, r2 = df.loc[i1], df.loc[i2]
        delta_c = abs(r1['count'] - r2['count'])
        
        if delta_c > min_delta_c:
            if r1['count'] > r2['count']:
                part_res.append([i1, i2, r1['seq'], r2['seq'], r1['count'], r2['count'], delta_c, k])
            else:
                part_res.append([i2, i1, r2['seq'], r1['seq'], r2['count'], r1['count'], delta_c, k])
        else:
            pass

    # Turn into a pandas dataframe and output the result as a csv
    out_filename = 'paired_partial_datasets/paired_dataset_lk_' + str(k) + '.csv'
    pd.DataFrame(part_res).to_csv(out_filename, header=False, index=False)

In [9]:
df = pd.read_csv(dataset_file, index_col=0)
df_kings = pd.read_csv(local_kings_file, index_col=0)
king_indexes = df_kings['lk'].tolist()

In [16]:
with Pool(5) as p:
    p.map(assemble_pairs, king_indexes)